# Intermediate SQL

## Join expressions

Previously, we combined information from several multiple queries using the cartesian product operator (except when we used set operations). In this section we introduce a number of <strong>Join operations</strong> that allow us to express some queries which are harder using the cartesian operator. As before, we will continue using the UNI database for now. Run the scripts in the folder before.

### The ``NATURAL JOIN``

Consider the following SQL query which computes, for each student, the set of courses a student has taken. 

In [2]:
USE uni;
GO
-- the query
SELECT TOP 3 --restricting to first 3 obs
student.name, takes.course_id
FROM student, takes
WHERE student.ID = takes.ID;
GO

Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.005

name,course_id
Manber,239
Manber,319
Manber,362
Manber,493
Manber,571
Manber,642


Note that in the student and takes table, the matching condition required both <code>ID</code> variables to be the same.

The natural join operation operates on two relations and produces a relation as result. However, unlike the cartesian product, which concatenates each row of the first relation with all of the second, the <strong>natural join considers only the pairs of rows with the same value on the attributes which appear in the schema of both relations</strong>.

<code>Natural Join</code>s are not supported by MS SQL servers. The code in other servers goes like this.

```
student NATURAL JOIN takes;
```

MS SQL, probably for the best, forces you to pre-select the features you want to join the ralations on. Wrote, however, a small nested query which, given two tables, returns the common features. In this case, as expected, it is the ``ID`` feature.

In [10]:
-- checking shared features with a subquery
SELECT COLUMN_NAME AS common_feature
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'student' AND COLUMN_NAME IN (
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'takes'
);
GO

(1 row affected)

Total execution time: 00:00:00.053

common_feature
ID


Join Conditions<br>

As we saw before, the nice property of natural joins is that it identifies overlapping features for joining the data.

The <code>ON</code> keyword allows a general predicate for selecting which variables should be used for the join operation. The on condition is writen exactly as a where predicate. It also appears at the end of the join expression.<br>

In [17]:
-- the join
SELECT COUNT(*) AS row_number
FROM student
JOIN takes ON student.ID = takes.ID;
GO
-- which is the same as
SELECT COUNT(*) AS row_number
FROM student, takes
WHERE student.ID = takes.ID;
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.009

row_number
30000


row_number
30000


### `OUTER JOIN`

Suppose we wish to display a list of all students, displaying their id, name, dept_name, tot_cred, along with the courses that they have taken. The following query we used before does not work very well. Suppose that there is some student that takes no course and who, hence, cannot be found in the takes relation (via ID). This student will be dropped out from the resulting relation.

In [26]:
-- insert a dummy
IF NOT EXISTS (SELECT ID FROM student WHERE ID = '10049')
    BEGIN
        INSERT INTO student VALUES ('10049', 'Snow', 'Civil Eng.', 0)
    END;
    GO

-- we loose it with a natural join
SELECT *
FROM student
JOIN takes ON student.ID = takes.ID
WHERE student.ID = '10049';
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.009

ID,name,dept_name,tot_cred,ID,course_id,sec_id,semester,year,grade


More generally, some tuples in either or both relations being joined may be <em>lost</em> in this way. The outer join operation works in a manner similar to the join operations already studied but <strong>it preserves those tuples that would be lost in a join by creating tuples in the sult containing NULL values.</strong>


There are three forms of outer join:

1. The ``LEFT OUTER JOIN`` preserves tuples in the relation named to the left of the join operation.

In [27]:
SELECT *
FROM student
LEFT OUTER JOIN takes ON student.ID = takes.ID
WHERE student.ID = '10049';
GO

(1 row affected)

Total execution time: 00:00:00.006

ID,name,dept_name,tot_cred,ID,course_id,sec_id,semester,year,grade
10049,Snow,Civil Eng.,0,NULL,NULL,NULL,NULL,NULL,NULL


2. The `RIGHT OUTER JOIN` perserves tuple only in the relation named after the join operation

In [31]:
-- we miss it like this, since it does not exist in takes
SELECT *
FROM student
RIGHT OUTER JOIN takes ON student.ID = takes.ID
WHERE student.ID = '10049';
GO

-- but not like this
SELECT *
FROM takes
RIGHT OUTER JOIN student ON student.ID = takes.ID
WHERE student.ID = '10049';
GO

(0 rows affected)

(1 row affected)

Total execution time: 00:00:00.010

ID,name,dept_name,tot_cred,ID,course_id,sec_id,semester,year,grade


ID,course_id,sec_id,semester,year,grade,ID,name,dept_name,tot_cred
NULL,NULL,NULL,NULL,NULL,NULL,10049,Snow,Civil Eng.,0


3. ``FULL OUTER JOIN`` preserves tuples in both relations

In [32]:
SELECT *
FROM student
FULL OUTER JOIN takes ON student.ID = takes.ID
WHERE student.ID = '10049';
GO

(1 row affected)

Total execution time: 00:00:00.004

ID,name,dept_name,tot_cred,ID,course_id,sec_id,semester,year,grade
10049,Snow,Civil Eng.,0,NULL,NULL,NULL,NULL,NULL,NULL


### ``INNER JOIN``

In contrast, the join operation that do not preserve nonmatched tuples are called inner join operations. It basically returns a table containing tuples in both relations.

In [82]:
-- not missing
SELECT TOP 6 *
FROM student
INNER JOIN takes on student.ID = takes.ID;
GO

-- missing is still...missing
SELECT TOP 6 *
FROM student
INNER JOIN takes ON student.ID = takes.ID
WHERE student.ID = '10049';
GO

(6 rows affected)

(0 rows affected)

Total execution time: 00:00:00.010

ID,name,dept_name,tot_cred,ID,course_id,sec_id,semester,year,grade
1000,Manber,Civil Eng.,39,1000,239,1,Fall,2006,C
1000,Manber,Civil Eng.,39,1000,319,1,Spring,2003,B+
1000,Manber,Civil Eng.,39,1000,362,1,Fall,2005,B+
1000,Manber,Civil Eng.,39,1000,493,1,Spring,2010,A-
1000,Manber,Civil Eng.,39,1000,571,1,Spring,2004,C+
1000,Manber,Civil Eng.,39,1000,642,1,Fall,2004,C-


ID,name,dept_name,tot_cred,ID,course_id,sec_id,semester,year,grade


### `Cross Join`

The SQL CROSS JOIN produces a result set which is the number of rows in the first table multiplied by the number of rows in the second table if no WHERE clause is used along with CROSS JOIN.This kind of result is called as Cartesian Product.  


In [45]:
SELECT COUNT(student.ID) AS [count]
FROM student
CROSS JOIN takes;
GO

SELECT COUNT(ID) AS [count]
FROM student;
GO

SELECT COUNT(ID) AS [count]
FROM takes;
GO

SELECT 

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.016

count
60060000


count
2002


count
30000


## Views

It is not always desirable for all users to see the entire set of relations in the database. For example, a university worker may need to access the students-related tables but we might want her to not have access to the salaries.

Aside from security reasons, we may wish to create a personalized collection of virtual relation that s better matched to a certain user's intuition of the structure of the database. For example, we may want to have a list of all course sections offered by the physics department in the fall of 2007 with the building and room number of each section.

In [51]:
-- with explicitly joins
SELECT course.course_id, course.title, section.building, section.room_number 
FROM course
INNER JOIN section ON course.course_id = section.course_id
WHERE semester = 'Fall' AND dept_name = 'Physics' AND [year] = '2007';
GO
-- alternatively
SELECT course.course_id, course.title, section.building, section.room_number 
FROM course, section
WHERE semester = 'Fall' AND 
      dept_name = 'Physics' AND 
      section.[year] = '2007' AND
      course.course_id = section.course_id;
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.013

course_id,title,building,room_number
612,Mobile Computing,Lamberton,143


course_id,title,building,room_number
612,Mobile Computing,Lamberton,143


It is possibe to compute and store the results of queries such as this and then make them stored relations available to users. However, if we did so, and the underlying data in the relations instructor, course, or section changed, the stored query results would then no longer match the result of reexecuting the query on the relations. In general, it is a bad idea to compute and store query results such as those in the above examples.

<br>

Instead, SQL allows us a "virtual relatio" to be defined by a query and the relation conceptually contains the result of the query. The virtual relation is not precomputed and stored but instead is computed by executing the query whenever the virtual relation is used. Namely, using ``VIEW``s.

<br>

We define a view in SQL using the ``CREATE VIEW`` command. More precisely

```
create view v as <query expression>;
GO
```
Using the query from above...

In [59]:
CREATE VIEW physics_fall_2007 AS 
SELECT course.course_id, course.title, section.building, section.room_number 
FROM course
INNER JOIN section ON course.course_id = section.course_id
WHERE semester = 'Fall' AND dept_name = 'Physics' AND [year] = '2007';
GO

: Msg 2714, Level 16, State 3, Procedure physics_fall_2007, Line 1
There is already an object named 'physics_fall_2007' in the database.

Total execution time: 00:00:00.001

In [54]:
SELECT * 
FROM physics_fall_2007;
GO


(1 row affected)

Commands completed successfully.

Total execution time: 00:00:00.004

course_id,title,building,room_number
612,Mobile Computing,Lamberton,143


In [57]:
-- Another example
CREATE VIEW faculty AS 
SELECT ID, [name], dept_name
FROM instructor;
GO

SELECT TOP 6 * 
FROM faculty;
GO

-- drop it
DROP VIEW faculty;
GO

Commands completed successfully.

(6 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.007

ID,name,dept_name
10076,Duan,Civil Eng.
10204,Mediratta,Geology
10454,Ugarte,Pol. Sci.
10527,Kieras,Physics
10693,Zabary,Statistics
10834,More,Geology


### Using ``VIEW``s in queries

Once we have defined a view, we can use the view name to refer to the virtual relation and to make queries on this virtual relation.

In [62]:
SELECT COUNT(*) as [n]
FROM physics_fall_2007;

-- drop
DROP VIEW physics_fall_2007;

(1 row affected)

Total execution time: 00:00:00.018

n
1


In [65]:
-- department salary
CREATE VIEW department_total_salary (dept_name, total_salary) AS
SELECT dept_name, SUM (salary) AS total_salary
FROM instructor
GROUP BY dept_name;
GO

SELECT * FROM department_total_salary;
GO

-- drop
DROP VIEW department_total_salary;
GO


Commands completed successfully.

(20 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.012

dept_name,total_salary
Accounting,600880.37
Astronomy,746093.08
Athletics,1139516.99
Biology,876600.50
Civil Eng.,667023.00
Comp. Sci.,911917.63
Cybernetics,994407.27
Elec. Eng.,934672.96
English,955379.20
Finance,743333.38


## Transactions

A transaction consists of a sequence of query and/or update statements. The SQL standard specifies that a transaction begins implicitly when an SQL statement is executed. One of the following SQL statements must end the transaction:

- **commit work**; commits the current transaction, that is, it makes the updates performed by the transaction become permanent in the database. After the transaction is committed, a new transaction is automatically started. Once a transaction is commited it cannot be reversed by rollback.
- **rollback** causes the current transaction to be rolled back; that is, it undoes all the updates performed by the SQL statements in the transaction. Thus, the tabase state is restored to what it was before the first statement of the transaction execution. **Rollback** is particularly useful if some error is detected during the execution of the transaction.

For instance, consider a banking application where we need to transfer money from one bank account to another in the same bank. To do so, we need to update two account balances, subtracting the amount transferred from one, and adding it to the other. If the system crashes after subtracting the amount from the first account but before adding it to the second account, the bank balances will be inconsistent. A similar problem occurs if the second account is credited before subtracting the amount from the first account and the system crashes just after crediting the amount.

As another example, consider our running example of a university application. We assume that the attribute tot cred of each tuple in the student relation is kept up-to- date by modifying it whenever the student successfully completes a course. To do so, whenever the takes relation is updated to record successful completion of a course by a student (by assigning an appropriate grade), the corresponding student tuple must also be updated. If the application performing these two updates crashes after one update is performed, but before the second one is performed, the data in the database will be inconsistent.

Applying the notion of transactions to the above applications, the update statements should be executed as a single transaction. An error while a transaction executes one of its statements would result in undoing the effects of the earlier statements of the transaction so that the database is not left in a partially updated state.

Below we have an example of a data insertion with explicit transaction control. In this case, one of the insertions triggers an error due to a violation of an integrity constraint. We run the code under a try-catch framework, if an error is caught we roll back so that the database returns to the original state, else we commit it.

In [80]:
-- check before
SELECT COUNT(*) AS n FROM student;
GO

-- the transaction
BEGIN TRANSACTION;
    -- Try to run the code and catch the error
    BEGIN TRY 
        INSERT INTO student VALUES ('14', 'Anne', NULL, 60); -- This is fine
        INSERT INTO student VALUES ('9002', 'Jane', NULL, -75); -- This violates integritiy constraint of stricly positive total credits
    END TRY
-- check if any errors occured, if yes and there are more than one transactions, rollback
BEGIN CATCH
    SELECT 
        ERROR_NUMBER() AS ErrorNumber,
        ERROR_SEVERITY() AS ErrorSeverity,
        ERROR_STATE() AS ErrorState,
        ERROR_PROCEDURE() AS ErrorProcedure,
        ERROR_LINE() AS ErrorLine,
        ERROR_MESSAGE() AS ErrorMessage;
    -- more than one error, rollback
    IF @@TRANCOUNT > 0
        ROLLBACK TRANSACTION;
END CATCH;
-- no error and more than 0 transactions, commit it
IF @@TRANCOUNT > 0 -- Returns the number of BEGIN TRANSACTION statements that have occurred on the current connection.
    COMMIT TRANSACTION;
GO

-- check after: If code goes well, due to the roll back, both should look the same
SELECT COUNT(*) AS n FROM student;
GO

(1 row affected)

(1 row affected)

(0 rows affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.006

n
2006


ErrorNumber,ErrorSeverity,ErrorState,ErrorProcedure,ErrorLine,ErrorMessage
547,16,0,NULL,7,"The INSERT statement conflicted with the CHECK constraint ""CK__student__tot_cre__3A81B327"". The conflict occurred in database ""uni"", table ""dbo.student"", column 'tot_cred'."


n
2006


Here we do the same, but now both insertions are fine.

In [81]:
-- check before
SELECT COUNT(*) AS n FROM student;
GO

-- the transaction
BEGIN TRANSACTION;
    -- Try to run the code and catch the error
    BEGIN TRY 
        INSERT INTO student VALUES ('14', 'Anne', NULL, 60); -- This is fine
        INSERT INTO student VALUES ('9002', 'Jane', NULL, 75); -- This violates integritiy constraint of stricly positive total credits
    END TRY
-- check if any errors occured, if yes and there are more than one transactions, rollback
BEGIN CATCH
    SELECT 
        ERROR_NUMBER() AS ErrorNumber,
        ERROR_SEVERITY() AS ErrorSeverity,
        ERROR_STATE() AS ErrorState,
        ERROR_PROCEDURE() AS ErrorProcedure,
        ERROR_LINE() AS ErrorLine,
        ERROR_MESSAGE() AS ErrorMessage;
    -- more than one error, rollback
    IF @@TRANCOUNT > 0
        ROLLBACK TRANSACTION;
END CATCH;
-- no error and more than 0 transactions, commit it
IF @@TRANCOUNT > 0
    COMMIT TRANSACTION;
GO

-- check after: If code goes well, due to the roll back, both should look the same
SELECT COUNT(*) AS n FROM student;
GO


(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.009

n
2006


n
2008


## Integrity constraints

Integrity constraints ensure that changes made to the database by unauthorized users do not result in a loss of data consistency. **Integrity constraints are usually identified as part of the database schema design process and declared as part of the creation or update of a relation**.

Among other, SQL allows the following integrity constraints:

1. ``NOT NULL``
```
CREATE TABLE(
name VARCHAR(20) NOT NULL
budget NUMERIC(12,2) NOT NULL
)
```
2. ``UNIQUE``
```
CREATE TABLE(
name VARCHAR(20) NOT NULL
budget NUMERIC(12,2) NOT NULL
CONSTRAINT unique_name UNIQUE(name)  
)
```
3. ``CHECK(<predicate>)``

```
CREATE TABLE(
name VARCHAR(20) NOT NULL
budget NUMERIC(12,2) NOT NULL
CONSTRAINT unique_name UNIQUE(name)  
CONSTRAINT budget_range CHECK (budget BETWEEN 0 and 100000000)
)
```
Or 
```
CREATE TABLE(
name VARCHAR(20) NOT NULL
semester VARCHAR(20)
budget NUMERIC(12,2) NOT NULL
CONSTRAINT unique_name UNIQUE(name)  
CONSTRAINT semester_check CHECK (semester IN ('fall', 'summer', 'autumn', 'winter'))
)
```

## Referential Integrity

Often we wish to ensure that a value which appears in one relation, the <em>referencing relation</em>, for a given set of attributes also appears for a certain set of attributes in another relation (the <em>referenced relation</em>). We saw before how to set referential integrity constraints, namely, foreign keys.

When a referential-integrity constraint is violated, the normal procedure is to reject the action that caused the violation (i.e., the transaction performing the update action is rolled back).

However, a <strong>foreign key clause can specify that if a delete or update action on the referenced relation violates the constraint, then, instead of rejecting the action, the system must take steps to change the tuple in the referencing relation to restore the constraint</strong>.

```
create table course ( ...
    foreign key (dept name) references department,
        on delete cascade
        on update cascade,
... );

```

Because of the <strong>on delete cascade</strong> associated with the foreign-key declaration, if a delete of a tuple in the <em>department</em> results in this <em>referential-integrity constraint being violated, the system does not reject the delete</em>. Instead, <em>the delete cascades</em> to the course relation, that is, that tuple will also be deleted there.

Similarly, the system does not reject an update to a filed referenced by the constrant if it violates the constraint, instead <em>the system updates the field detp_name in the referencing tuples in course to the new value as well</em>.

SQL also allows the foreign key clause to specify actions other than cascade, if the constraint is violated: The referencing field (here, dept name) <strong>can be set to null (by using set null in place of cascade)</strong>, or <strong>to the default value for the domain (by using set default)</strong>.

## Integrity constraint violation during a transaction

Transactions may consist of several changes to the database and integrity constraints may be temporarily violated after one of these steps but a later step may remove the violation.

To handle such situaitons, the SQL standard allows a clause <strong>initially deferred</strong> to be added to a constraint specification; the constraint would then be checked at the end of a transaction and not at intermediate steps.

A constrain can alternatively be specified as <strong>deferrable</strong>, which means it is checked immediatly by default but can be deferred when desired.

For constraints declared as deferrable, executing a statement <strong>set constraints</strong> <em>constraint-list</em> <strong>deferred</strong> as part of a transaction causes the checking of the specified constraints to be deferred to the end of that transaction. Constraints which appear in the constraint list must, obviously, be named during their creation.

<br>

# Practice exercises from the book

(From chapter 4 of Silberchatz et al)

<br>

4.1. Consider the following SQL query that seeks to find a list of titles of all courses taught in Spring 2017 along with the name of the instructor

In [3]:
USE uni;
GO
-- no joins
SELECT instructor.name, course.title
FROM course, instructor, teaches
WHERE 
    course.course_id = teaches.course_id AND
    instructor.ID = teaches.ID AND 
    teaches.semester = 'Spring' AND
    teaches.year = '2007';
GO

-- With nested inner joins
SELECT [name], title
FROM course 
    INNER JOIN teaches ON  course.course_id = teaches.course_id
    INNER JOIN instructor ON instructor.ID = teaches.ID
WHERE  
    teaches.semester = 'Spring' AND
    teaches.year = '2007';
GO

Commands completed successfully.

(6 rows affected)

(6 rows affected)

Total execution time: 00:00:00.018

name,title
Lembr,The Music of the Ramones
DAgostino,Graph Theory
DAgostino,Visual BASIC
Gustafsson,Marine Mammals
Gustafsson,Plasma Physics
Ullman,Bankruptcy


name,title
Lembr,The Music of the Ramones
DAgostino,Graph Theory
DAgostino,Visual BASIC
Gustafsson,Marine Mammals
Gustafsson,Plasma Physics
Ullman,Bankruptcy


4.2. Write the following queries in SQL

a) Display a list of all instructors showing each instructor ID and the number of sections thaught. Make shure to show the number of sectons as 0 for instructors who have not thaught any sections. Your query should use outer join and no subqueries.<br>

In [6]:
SELECT instructor.ID, instructor.[name], COUNT(teaches.sec_id) AS section_n
FROM instructor
LEFT OUTER JOIN teaches ON teaches.ID = instructor.ID
GROUP BY instructor.ID, instructor.name;

(513 rows affected)

Total execution time: 00:00:00.005

ID,name,section_n
10076,Duan,0
10204,Mediratta,0
10454,Ugarte,0
10527,Kieras,0
10693,Zabary,0
10834,More,0
11055,Arnoux,0
11057,Robinson,0
11076,Prasad,0
11083,Kerridge,0


b) Write the same query as above, but using a scalar subquery and not using outerjoin

In [7]:
SELECT ID, (
    SELECT COUNT(*) AS section_n
    FROM teaches
    WHERE teaches.ID = instructor.ID
)
FROM instructor;
GO

(513 rows affected)

Total execution time: 00:00:00.006

ID,(No column name)
10076,0
10204,0
10454,0
10527,0
10693,0
10834,0
11055,0
11057,0
11076,0
11083,0


c) Display the list of all course sections offered in Spring 2008, along with the ID and name of each instructor teaching the section. If a section has more than one instructor, that section should appear as many times in the result as it has instructors. If a section does not have any instructor, it should still appear in the result with the instructor name set to '-'.

In [14]:

SELECT section.course_id, section.sec_id, instructor.ID, IIF(
    instructor.name IS NULL, 
    '-', 
    instructor.name
    ) AS instructor_name
FROM (section LEFT OUTER JOIN teaches ON section.sec_id = teaches.sec_id AND section.course_id = teaches.course_id)
    LEFT OUTER JOIN instructor ON teaches.ID = instructor.ID
WHERE section.semester = 'Spring' AND section.[year] = '2008';
GO

(9 rows affected)

Total execution time: 00:00:00.003

course_id,sec_id,ID,instructor_name
158,2,99052,Dale
237,1,99052,Dale
345,1,79081,Ullman
349,1,6569,Mingoz
362,3,6569,Mingoz
704,1,77346,Mahmoud
852,1,63287,Jaekel
962,1,22591,DAgostino
991,1,22591,DAgostino


d) Display the lsit of all departments, with the total number of instructors in each department, without using subqueries. Make sure to show departments that have no instructors and list those departments with an instructor count of zero.

In [15]:
SELECT department.dept_name, COUNT(instructor.ID) AS instructor_n
FROM department LEFT OUTER JOIN instructor ON department.dept_name = instructor.dept_name
GROUP BY department.dept_name;
GO

(20 rows affected)

Total execution time: 00:00:00.007

dept_name,instructor_n
Accounting,18
Astronomy,24
Athletics,31
Biology,28
Civil Eng.,23
Comp. Sci.,26
Cybernetics,25
Elec. Eng.,26
English,27
Finance,23


## Exercise from class

Be sure to run ``scripts/Create_PhotoSharing.sql`` and ``scripts/PhotoSharing_LoadRandomData.sql`` before doing the exercise.

The database in a nutshell


In [17]:
USE PhotoSharing;
GO

SELECT TABLE_NAME, COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS;
GO

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.032

TABLE_NAME,COLUMN_NAME
COMMENT,CID
COMMENT,PID
COMMENT,SCORE
COMMENT,TXT
COMMENT,UID
PICTURE,AUTHOR
PICTURE,IMG
PICTURE,PID
USERS,NAME
USERS,UID


1.

a) Write a SQL query that returns all users who have given a score of 4 or higher to 5 pictures or more. For each user, your query should return the user ID and the name.

In [29]:
-- with join clause
select USERS.[NAME], USERS.UID
from USERS left join COMMENT on USERS.UID = COMMENT.UID
where COMMENT.SCORE >= 4 
GROUP BY USERS.[NAME], USERS.UID
HAVING COUNT(*) >= 5;

-- without
select USERS.[NAME], USERS.UID
from USERS, COMMENT
where COMMENT.SCORE >= 4 AND USERS.UID = COMMENT.UID
GROUP BY USERS.[NAME], USERS.UID
HAVING COUNT(*) >= 5;


(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.010

NAME,UID
Zephr L. Sheppard,27
Galvin T. Oneal,28
Vincent C. Sheppard,84


NAME,UID
Zephr L. Sheppard,27
Galvin T. Oneal,28
Vincent C. Sheppard,84


b) A picture is considered highly rated if it received at least one score of 5 from a user other than its author. A caustious user is a user who commented only on highly rated pictures. A user who did not comment at all is also cautious. Write a SQL query that finds all cautious users. Your query should return a list of uid, name pairs.

In [44]:
-- users which commented only on highly rated pictures (solution with subqueries)
select distinct USERS.[NAME], USERS.UID
from USERS right outer join COMMENT on USERS.UID = COMMENT.UID
-- subquery for all picture ids which have more than 1 ratings of 5
where COMMENT.PID in ( 
    select COMMENT.PID
    from COMMENT, PICTURE
    where COMMENT.SCORE = 5 and COMMENT.PID = PICTURE.PID
    group by COMMENT.PID
    having count(*) > 1
)
-- union of the sets
union 
-- users who did not comment
select distinct USERS.[NAME], USERS.UID
from USERS, COMMENT
where USERS.UID not in (select COMMENT.UID from COMMENT); 

(24 rows affected)

Total execution time: 00:00:00.017

NAME,UID
Devin L. Boyd,1
Sloane F. Weaver,12
Courtney W. Bush,18
Kiona U. Avery,19
Elijah U. Hamilton,23
Zephr L. Sheppard,27
Chelsea V. Stokes,39
Mollie T. Levy,40
Malachi P. Pate,41
Adena E. Burch,48


2. A hacker found a way to break into the system and ran the following commands

```
insert into Comment(uid, pid, score, txt)
select x.uid, y.pid, 0 as score, 'worst picture I ever saw' as txt
from Users x, Picture y
where x.uid = y.author;
update Picture
set author = (select top 1 x.uid
from Comment x
where x.pid = Picture.pid
order by x.score desc);

```

b) Your task is to repair the database writing a set of SQL commands that put it in its original state knowing that there were no backups before the hackers intrusion. Use INSERT/UPDATE/DELETE COMMANDS knowing that previously the scores were between 1 and 5.

In [47]:
-- n rows with score = 0
select COUNT(*)
from COMMENT
where COMMENT.SCORE = 0;

-- author ids messed up are the same as those with score 0
update PICTURE
	set author =
		(select COMMENT.uid
		 from COMMENT, PICTURE
		 where COMMENT.score = 0 and COMMENT.pid = PICTURE.pid
		);

-- Removing the data added by the hacker
delete from COMMENT
    where COMMENT.SCORE = 0;

-- Check
select COUNT(*)
from COMMENT
where COMMENT.SCORE = 0;

(1 row affected)

The statement has been terminated.

(250 rows affected)

(1 row affected)

Total execution time: 00:00:00.012

(No column name)
250


(No column name)
0


: Msg 512, Level 16, State 1, Line 7
Subquery returned more than 1 value. This is not permitted when the subquery follows =, !=, <, <= , >, >= or when the subquery is used as an expression.

In [52]:
USE master;
GO
-- delete the databases
IF EXISTS (SELECT [name] FROM sys.databases WHERE [name] = 'uni')
    BEGIN   
        -- kill all connections active with the database
        ALTER DATABASE uni SET SINGLE_USER WITH ROLLBACK IMMEDIATE
        DROP DATABASE uni
    END;
GO

IF EXISTS (SELECT [name] FROM sys.databases WHERE [name] = 'PhotoSharing')
    BEGIN
        -- kill all connections active with the database
        ALTER DATABASE PhotoSharing SET SINGLE_USER WITH ROLLBACK IMMEDIATE
        DROP DATABASE PhotoSharing
    END;
GO
-- double-check
SELECT [name] FROM sys.databases;
GO

Commands completed successfully.

Nonqualified transactions are being rolled back. Estimated rollback completion: 0%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 100%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 0%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 100%.

(6 rows affected)

Total execution time: 00:00:06.377

name
master
tempdb
model
msdb
udemy
dbm_project


### Exercises from class 9: Microcredit - queries

Be sure to restore the backup file provided in class, ``data/microcredit.bak``. In my case, the following code, after moving the backup to the location of the server in a new folder, works. This code is auto generated by azure data studio.


In [2]:
IF NOT EXISTS (SELECT [name] FROM sys.databases WHERE [name] = 'MICROCREDIT')
    BEGIN
        -- restore the database from the backup file
        USE [master]
        BACKUP LOG [MICROCREDIT] TO  DISK = N'/var/opt/mssql/data/MICROCREDIT_LogBackup_2020-12-05_22-01-48.bak' WITH NOFORMAT, NOINIT,  NAME = N'MICROCREDIT_LogBackup_2020-12-05_22-01-48', NOSKIP, NOREWIND, NOUNLOAD,  NORECOVERY ,  STATS = 5
        RESTORE DATABASE [MICROCREDIT] FROM  DISK = N'/var/opt/mssql/temp-backups/microcredit.bak' WITH  FILE = 1,  MOVE N'MICROCREDIT' TO N'/var/opt/mssql/data/MICROCREDIT.mdf',  MOVE N'MICROCREDIT_log' TO N'/var/opt/mssql/data/MICROCREDIT_log.ldf',  NOUNLOAD,  STATS = 5
    END;
GO

USE MICROCREDIT;
GO

SELECT TABLE_NAME, COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS;
GO


Commands completed successfully.

Commands completed successfully.

(40 rows affected)

Total execution time: 00:00:00.045

TABLE_NAME,COLUMN_NAME
HISTORICDATES,HISTORICDATE
HISTORICDATES,HISTORICID
HISTORICDATES,LOAN
LOANS,DATEOFLOAN
LOANS,LIMITPAYMENTDATE
LOANS,LOAN_APPLICATION
LOANS,LOANID
LOAN_APPLICATIONS,DESCRIPTION
LOAN_APPLICATIONS,ENTREPRENEUR
LOAN_APPLICATIONS,LOAN


1. Insert new loan commitments with entrepreneurs investing in their own projects 10% of the total amount requested in the loan application with commitment date = loan application date.

In [6]:
-- before
select count(*)
FROM LOAN_COMMITMENTS;
GO

-- update
insert into LOAN_COMMITMENTS (COMMITMENTAMOUNT, COMMITMENTDATE, INVESTOR, LOAN_APPLICATION)
    select lapp.TOTALLOANAMOUNT * 1.10, lapp.LOANAPPLICATIONDATE, lapp.ENTREPRENEUR, lapp.LOANAPPLICATIONID
    from LOAN_APPLICATIONS as lapp;
GO

-- after
select count(*)
FROM LOAN_COMMITMENTS;
GO

(1 row affected)

(164 rows affected)

(1 row affected)

Total execution time: 00:00:00.019

(No column name)
273


(No column name)
437


2. Write a query to find if users are investing in their own projects and delete those commitments

In [7]:
-- before
select lc.INVESTOR
from LOAN_COMMITMENTS as lc, LOAN_APPLICATIONS as lp
where lc.LOAN_APPLICATION = lp.LOANAPPLICATIONID and lc.INVESTOR = lp.ENTREPRENEUR;
GO

-- delete
delete LOAN_COMMITMENTS
from USERS as u, LOAN_COMMITMENTS as lc, LOAN_APPLICATIONS as lp
where lc.LOAN_APPLICATION = lp.LOANAPPLICATIONID and lc.INVESTOR = lp.ENTREPRENEUR;
GO

-- after
select lc.INVESTOR
from LOAN_COMMITMENTS as lc, LOAN_APPLICATIONS as lp
where lc.LOAN_APPLICATION = lp.LOANAPPLICATIONID and lc.INVESTOR = lp.ENTREPRENEUR;
GO

(164 rows affected)

(164 rows affected)

(0 rows affected)

Total execution time: 00:00:00.061

INVESTOR
3
7
8
9
9
10
12
14
19
20


INVESTOR


3. Calculate the total amount of commitments that each loan application have received.

In [15]:
select LOAN_APPLICATION, sum(COMMITMENTAMOUNT) as total_commitment_amount
from LOAN_COMMITMENTS
group by LOAN_APPLICATION;
go

(159 rows affected)

Total execution time: 00:00:00.003

LOAN_APPLICATION,total_commitment_amount
43,291.3300
44,5320.4200
45,4090.9200
46,584.3200
47,4315.6800
48,1310.4700
49,1063.4400
50,3112.2000
51,514.5000
52,6043.5000


4. From the previous query select only the loan application sfor which commitments didn't overcome the initial amount requestd

In [20]:
-- with subqueries
select lc.LOAN_APPLICATION, sum(lc.COMMITMENTAMOUNT) as total_commitment_amount
from LOAN_COMMITMENTS as lc
group by lc.LOAN_APPLICATION
having sum(COMMITMENTAMOUNT) < (
    select lap.TOTALLOANAMOUNT
    from LOAN_APPLICATIONS as lap
    where lap.LOANAPPLICATIONID = lc.LOAN_APPLICATION
);
go

-- with a join expression
select lc.LOAN_APPLICATION, sum(lc.COMMITMENTAMOUNT) as total_commitment_amount
from LOAN_COMMITMENTS as lc inner join  LOAN_APPLICATIONS as lap on lc.LOAN_APPLICATION = lap.LOANAPPLICATIONID
group by lc.LOAN_APPLICATION, lap.TOTALLOANAMOUNT
having sum(lc.COMMITMENTAMOUNT) < lap.TOTALLOANAMOUNT;
go

(157 rows affected)

(157 rows affected)

Total execution time: 00:00:00.007

LOAN_APPLICATION,total_commitment_amount
43,291.3300
44,5320.4200
45,4090.9200
46,584.3200
47,4315.6800
48,1310.4700
49,1063.4400
50,3112.2000
51,514.5000
52,6043.5000


LOAN_APPLICATION,total_commitment_amount
43,291.3300
44,5320.4200
45,4090.9200
46,584.3200
47,4315.6800
48,1310.4700
49,1063.4400
50,3112.2000
51,514.5000
52,6043.5000


5. Update commitments from the investor that have the maximum commited amount for each loan application such that the loan applications from the previous query reach the initial amount requested.

6. For each loan payment generate proportional reimbursements to the investors that made commitments to the original loan application

In [26]:
insert into REIMBURSEMENTS
select l.LOANID, lc.INVESTOR, getdate(),
lc.COMMITMENTAMOUNT/ (select sum(COMMITMENTAMOUNT) from LOAN_COMMITMENTS
where LOAN_APPLICATION = l.LOAN_APPLICATION) * p.PAYEDAMOUNT
from PAYMENTS p, LOANS l, LOAN_COMMITMENTS lc
where p.LOAN = l.LOANID and l.LOAN_APPLICATION = lc.LOAN_APPLICATION


(388 rows affected)

Total execution time: 00:00:00.048

7. Delete all risk tuples where there is no relationship in the past (status of loan application in (‘Conceeded’, ’Payed’)) between investor and entrepreneur

In [27]:
delete from RISK
where not exists (select 1 
                  from LOAN_APPLICATIONS la, LOAN_COMMITMENTS lc 
                  where la.LOANAPPLICATIONID = lc.LOAN_APPLICATION
                    and RISK.INVESTOR = LC.INVESTOR AND RISK.ENTREPRENEUR = LA.ENTREPRENEUR
                    and la.STATUS in ('Conceeded', 'Payed'));


(130 rows affected)

Total execution time: 00:00:00.020

In [29]:
-- Remove the database
USE master;
GO
IF EXISTS (SELECT [name] FROM sys.databases WHERE [name] = 'MICROCREDIT')
    BEGIN 
        DROP DATABASE microcredit
    END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.283